In [4]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [6]:
# specify the url strings for the company's job posting website
url1 = 'https://www.accenture.com/us-en/careers/jobsearch?jk=&sb=1&vw=0&is_rj=0&pg='
url2 = ''

# get the first page
driver=webdriver.Chrome(options = chrome_options)
URL = url1 + "1" + url2
driver.get(URL)
time.sleep(1)
soup = BeautifulSoup(driver.page_source, 'html.parser')
s = soup.find('a', {"class" : "cmp-jobs-results__title"}).text

# get total number of jobs and calculate total number of pages
total_count = int(s.replace(")", "").split("(")[1])
print(total_count)
total_page = int(total_count / 9 + 1)
print(total_page)

791
88


In [7]:
def accenture_get_jobs(url1, url2, totalPage):
    """
    retrieve job titles and job links from each page
    """
    title = []
    link = []
    page_num = 0

    driver=webdriver.Chrome(options = chrome_options)
    URL = url1 + str(page_num+1) + url2
    driver.get(URL)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup.find('a', {"class" : "cmp-jobs-results__title"}).tag
    
    # loop through all pages
    while page_num <= totalPage :
        driver.quit()
        
        for t in soup.find_all("a", {"class": "cmp-teaser__title-link"}):
            if "www.accenture.com/us-en/careers" in t['href']:
                link.append(t['href'])
                title.append(t.find('h3', {"class": "cmp-teaser__title"}).text)

        page_num += 1
        driver=webdriver.Chrome(options = chrome_options)
        URL = url1 + str(page_num+1) + url2
        
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit() 
    return title, link

In [8]:
job_title, job_link = accenture_get_jobs(url1, url2, total_page)

In [9]:
def remove_char(s):
  s_new = s.replace('\n', ' ').replace('\xa0', ' ')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

In [10]:
# create a dataframe that contains job titles and links for all job categories
df_title_link = pd.DataFrame(zip(job_title, job_link), columns=['JOB_TITLE', 'JOB_LINK'])

# drop the duplicates
df_title_link = df_title_link.drop_duplicates()

## Extract job descriptions and qualifications for each job

In [11]:
# retrieve job qualifications and descriptions
def job_description(title, link):
    qualifications = []
    description = []
    jobtitle = []
    joblink = []

    driver=webdriver.Chrome('chromedriver',options=chrome_options)
    for i in range(len(link)):
        URL=link[i]
        driver.get(URL)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        s = ''
        d = ''
        
        try:
          tag = soup.find('h2', text="Qualifications")
          try:
              tags = tag.findNext('ul').parent.find_all('ul')
              for t in tags:
                s = s + " " + t.text
          except: pass
        except: pass

        # retrieve job descriptions
        try:
          tag = soup.find('div', {'class': "description-content"})
          if tag:
            d = tag.text
        except: pass
        
        qualifications.append(s)
        description.append(d)   
        jobtitle.append(title[i])
        joblink.append(link[i])
        
    driver.quit()            
 
    return jobtitle, link, qualifications, description

In [12]:
# retrieve the qualification for each job.
title, link, qual, descrp = job_description(df_title_link['JOB_TITLE'].values, df_title_link['JOB_LINK'].values)

In [13]:
def remove_char(s):
  s_new = s.replace('\n', ' ').replace('\xa0', ' ')
  while '  ' in s_new:
    s_new = s_new.replace('  ', ' ')
  return s_new

qual_cleaned = [remove_char(q) for q in qual]
descrp_cleaned = [remove_char(d) for d in descrp]

In [14]:
#create a dataframe from qualifications and descriptions
df = pd.DataFrame(zip(title, qual_cleaned, link, descrp_cleaned))
df.columns = ['TITLE', 'QUALIFICATIONS', 'LINK', 'DESCRIPTION']
df['COMPANY'] = 'Accenture'
df = df.iloc[:, [4, 0, 1, 2, 3]]

In [15]:
# look for empty QUALIFICATION entries
def get_empty(df, colname) :
  empty_idx=[]
  for i in range(len(df[colname])):
    count = len(df[colname][i])
    if count<=10 : empty_idx.append(i)
    else: pass
  return empty_idx

In [16]:
empty_mini_q = get_empty(df, 'QUALIFICATIONS')
#print(len(empty_mini_q))

# drop the empty Qualification entries
df.drop(empty_mini_q, inplace=True)

#remove the duplicated jobs
df_nodup = df.drop_duplicates()
print("There are {} jobs from Accenture.".format(df_nodup.shape[0]))

# output to a file
df_nodup.to_csv('accenture_all_jobs_cleaned.csv')